In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

#Create dummy variables
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

## Concatenate the Data Frames

X=pd.concat([X,geography,gender],axis=1)

## Drop Unnecessary columns
X=X.drop(['Geography','Gender'],axis=1)

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid

In [ ]:
def create_model(layers, activation, drop):
  model=Sequential()
  #here the i is my index where the nodes is my values
  #if we donot give the parameter of the kernel_initializer so by default it selects glorot_uniform
  for i, nodes in  enumerate(layers):
    if i==0:
      model.add(Dense(units=nodes, input_dim=X_train.shape[1]))
      #here activation could be given seperately or could have been given by parameter in defining the layer
      model.add(Activation(activation))
      model.add(Dropout(drop))
    else:
      model.add(Dense(units=nodes))
      model.add(Activation(activation))
      model.add(Dropout(drop))
    
    #this is for the output
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model



In [ ]:
lst=[[20],[10,20],[10,20,30]]
for i,j in enumerate(lst):
  print(i,j)

0 [20]
1 [10, 20]
2 [10, 20, 30]


In [ ]:
#Keras has a scikit learn wrapper ( KerasClassifier ) that enables us to include K-fold cross validation in our 
#Keras code. The KerasClassifier expects one of its arguments to be a function, so we need to build that function.
#The purpose of this function is to build the architecture of our ANN
model = KerasClassifier(build_fn=create_model, verbose=2)

In [ ]:
from sklearn.model_selection import KFold
k_cv=KFold(n_splits=10,shuffle=True, random_state=100)

In [ ]:
layer = [[20], [40, 20], [45, 30, 15], [50,40,10]]
activations = ['sigmoid', 'relu']
p=[0.3,0.7,0.9]
param_grid = dict(layers=layer, activation=activations, drop=p, batch_size = [256, 128], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)

In [ ]:
grid

GridSearchCV(cv=5, error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f123e2d7090>,
             iid='deprecated', n_jobs=None,
             param_grid={'activation': ['sigmoid', 'relu'],
                         'batch_size': [256, 128], 'drop': [0.3, 0.7, 0.9],
                         'epochs': [30],
                         'layers': [[20], [40, 20], [45, 30, 15],
                                    [50, 40, 10]]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
grid_result = grid.fit(X_train, y_train)

Streaming output truncated to the last 5000 lines.
25/25 - 0s - loss: 0.8289 - accuracy: 0.4919
Epoch 2/30
25/25 - 0s - loss: 0.6747 - accuracy: 0.6122
Epoch 3/30
25/25 - 0s - loss: 0.6061 - accuracy: 0.6703
Epoch 4/30
25/25 - 0s - loss: 0.5552 - accuracy: 0.7277
Epoch 5/30
25/25 - 0s - loss: 0.5335 - accuracy: 0.7534
Epoch 6/30
25/25 - 0s - loss: 0.5153 - accuracy: 0.7673
Epoch 7/30
25/25 - 0s - loss: 0.4979 - accuracy: 0.7759
Epoch 8/30
25/25 - 0s - loss: 0.4990 - accuracy: 0.7809
Epoch 9/30
25/25 - 0s - loss: 0.4880 - accuracy: 0.7875
Epoch 10/30
25/25 - 0s - loss: 0.4786 - accuracy: 0.7911
Epoch 11/30
25/25 - 0s - loss: 0.4738 - accuracy: 0.7998
Epoch 12/30
25/25 - 0s - loss: 0.4686 - accuracy: 0.7983
Epoch 13/30
25/25 - 0s - loss: 0.4701 - accuracy: 0.8027
Epoch 14/30
25/25 - 0s - loss: 0.4611 - accuracy: 0.8009
Epoch 15/30
25/25 - 0s - loss: 0.4607 - accuracy: 0.8034
Epoch 16/30
25/25 - 0s - loss: 0.4616 - accuracy: 0.8003
Epoch 17/30
25/25 - 0s - loss: 0.4514 - accuracy: 0.8066


In [ ]:
print([grid_result.best_score_,grid_result.best_params_])

[0.8526250004768372, {'activation': 'relu', 'batch_size': 128, 'drop': 0.3, 'epochs': 30, 'layers': [40, 20]}]


In [ ]:
pred_y = grid.predict(X_test)
y_pred = (pred_y > 0.5)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [ ]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,y_pred)

In [ ]:
score

0.8605

In [ ]:
cm

array([[1533,   62],
       [ 217,  188]])